# Using Web Search Tool for Real-Time Information

## Web Search Integration

### Why Web Search?
- **Real-Time Data**: Access current information not in documents
- **Broader Knowledge**: Search the entire web, not just ingested documents
- **Dynamic Updates**: Information updates automatically

### Implementation
We'll use LlamaStack's built-in web search tool (`builtin::websearch`) which:
- Searches the web using search APIs (e.g., Tavily)
- Returns relevant web pages and snippets
- Integrates seamlessly with agents

### Setup Requirements
- **API Key**: Tavily search API key (set as environment variable)
- **Provider Data**: Pass API key to LlamaStackClient for web search access

In [1]:
# Install notebook dependencies. 
# Will take a while to download and install numerous dependencies. 
# Wait until it finishes before proceeding
%pip install llama_stack_client==0.2.22

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Python stdlib imports
import os
import json
from datetime import date, datetime, timedelta
import re
import logging

# Suppress verbose and noisy HTTP logs
logging.getLogger("httpx").setLevel(logging.WARNING)

# Llamastack imports
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types import UserMessage
from typing import cast, Iterator
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

import rich

In [3]:
# LlamaStack service URL (in-cluster)
LLAMASTACK_URL = "http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321"

# Get Tavily search API key from environment variable
# Tavily is a search API that provides web search capabilities
tavily_search_api_key = os.getenv('TAVILY_SEARCH_API_KEY')

# Configure provider data for web search
# If API key is available, pass it to enable web search functionality
if tavily_search_api_key is None:
    provider_data = None  # Web search will not be available
else:
    # provider_data: Configuration for external service providers
    # tavily_search_api_key: API key for Tavily search service
    provider_data = {"tavily_search_api_key": tavily_search_api_key}

# Reinitialize client with provider data for web search
# provider_data enables the client to use external services like Tavily
client = LlamaStackClient(
    base_url=LLAMASTACK_URL,
    provider_data=provider_data  # Enables web search if API key is provided
)

models = client.models.list()
model_id = next(m.identifier for m in models if m.model_type == "llm")

In [4]:
# Verify that tavily search is available as a tool in Llamastack
client.toolgroups.list()

[ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch')]

In [52]:
agent = Agent(
            client=client,
            model=model_id,
            instructions="You are a helpful assistant that provides real-time responses using the websearch tool.",
            tools=["builtin::websearch"],
            sampling_params={
                "max_tokens": 1512,
                "temperature": 0.3
            },
        )

#query = ["What is the latest news from the National Stock Exchange (NSE) of India?"] 
query = ["What is the latest exchange rate between USD and INR?"]

session_id = agent.create_session("tavily-session")
for prompt in query:
    rich.print(f"Processing user query: {prompt}")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=True
    )

for log in EventLogger().log(response):
    log.print()

Processing user query: What is the latest exchange rate between USD and INR?

inference> 
tool_execution> Tool:brave_search Args:{'query': 'latest exchange rate USD to INR'}
tool_execution> Tool:brave_search Response:{"query": "latest exchange rate USD to INR", "top_k": [{"url": "https://www.compareremit.com/todays-best-dollar-to-rupee-exchange-rate/", "title": "USD to INR - Today's Best US Dollar to Rupee Exchange ...", "content": "As of today, the USD to INR exchange rate is INR 84.27 INR for 1 US dollar. This rate marks a continued period of strength for the dollar against the Indian", "score": 0.9998233, "raw_content": null}, {"url": "https://wise.com/us/currency-converter/usd-to-inr-rate?amount=1", "title": "1 US dollar to Indian rupees Exchange Rate. Convert USD ...", "content": "USD to INR exchange rates today ; 1 USD, 89.86 INR ; 5 USD, 449.31 INR ; 10 USD, 898.62 INR ; 20 USD, 1,797.24 INR.", "score": 0.9996018, "raw_content": null}, {"url": "https://www.bookmyforex.com/currency-converter/usd-to-inr/", "title": "USD to INR | Convert US Dollar to Indian 